#Loading and Processing Data 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Dataset download from kaggle (https://www.kaggle.com/williamscott701/memotion-dataset-7k)
#Please download a kaggle.json issued against your kaggle account to run this notebook

!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d williamscott701/memotion-dataset-7k
!unzip /content/memotion-dataset-7k.zip
!rsync --info=progress2 '/content/drive/My Drive/2000_data.zip' '/content/'
!unzip '/content/2000_data.zip'

In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from tqdm import tqdm,tqdm_notebook
from sklearn.metrics import f1_score,accuracy_score
import math

In [ ]:
train=pd.read_csv('/content/Final_train.csv')
val=pd.read_csv('/content/Final_val.csv')
test=pd.read_csv('/content/final_test.csv')
print(train.shape)
print(val.shape)
print(test.shape)

# output:
# (5943, 8)
# (1049, 8)
# (1878, 4)

Transforming data labels

In [ ]:
# Converting removing 'very' from labels as suggested by the organizers of the competition

for i in range(train.shape[0]):
  if train.iloc[i,7]=='neutral':
    train.iloc[i,7]=0
  elif train.iloc[i,7]=='positive' or train.iloc[i,7]=='very_positive':
    train.iloc[i,7]=1
  else :
    train.iloc[i,7]=2    
for i in range(val.shape[0]):
  if val.iloc[i,7]=='neutral':
    val.iloc[i,7]=0
  elif val.iloc[i,7]=='positive' or val.iloc[i,7]=='very_positive':
    val.iloc[i,7]=1
  else :
    val.iloc[i,7]=2   

Dropping unneccessary columns and keeping only text and image file name columns

In [ ]:
data_train=train.iloc[:,[2,7,0]]
data_val=val.iloc[:,[2,7,0]]
data_train.columns=[0,1,2]
data_val.columns=[0,1,2]
data_test=test.loc[:,['corrected_text','Image_URL','Image_name']]

Regex Transformations

In [ ]:
import re

def process(data):
  for i in tqdm(range(data.shape[0])):
    eg=re.sub('[^a-zA-Z]',' ',data.iloc[i,0])
    #eg=re.sub('(?!^)([A-Z][a-z]+)', r' \1', eg).lower()
    #eg=re.sub(r'^"|"$', '', eg)
    eg=" ".join(eg.lower().split())
    #eg=eg.split()
    #ps=PorterStemmer()
    #eg=[word for word in eg if not word in set(stopwords.words('english'))]
    #eg=" ".join(eg)
    
    data.iloc[i,0]=eg
  return data  


data_train=process(data_train.copy())
data_val=process(data_val.copy())
data_test=process(data_test.copy())



# Model Training and Inference

Installing Transformers library to use Roberta Model

In [ ]:
!pip install transformers

Importing libraries

In [ ]:
import os
import h5py
import pprint
import pandas as pd 
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
import os 
import re
from tqdm import tqdm_notebook,tqdm
from sklearn.metrics import f1_score
from transformers import *

Hand Crafted Label Encoding (external libraries can also be used here for achieving same encoding) 

In [ ]:

for i in range(train.shape[0]):
  if train.iloc[i,3]=='hilarious':
    train.iloc[i,3]=3
  if train.iloc[i,3]=='very_funny':
    train.iloc[i,3]=2  
  if train.iloc[i,3]=='funny':
    train.iloc[i,3]=1
  if train.iloc[i,3]=='not_funny':
    train.iloc[i,3]=0

  if train.iloc[i,4]=='very_twisted':
    train.iloc[i,4]=3
  if train.iloc[i,4]=='twisted_meaning':
    train.iloc[i,4]=2  
  if train.iloc[i,4]=='general':
    train.iloc[i,4]=1
  if train.iloc[i,4]=='not_sarcastic':
    train.iloc[i,4]=0    

  if train.iloc[i,5]=='hateful_offensive':
    train.iloc[i,5]=3
  if train.iloc[i,5]=='very_offensive':
    train.iloc[i,5]=2  
  if train.iloc[i,5]=='slight':
    train.iloc[i,5]=1
  if train.iloc[i,5]=='not_offensive':
    train.iloc[i,5]=0    

  if train.iloc[i,6]=='motivational':
    train.iloc[i,6]=1
  if train.iloc[i,6]=='not_motivational':
    train.iloc[i,6]=0    

for i in range(val.shape[0]):
  if val.iloc[i,3]=='hilarious':
    val.iloc[i,3]=3
  if val.iloc[i,3]=='very_funny':
    val.iloc[i,3]=2  
  if val.iloc[i,3]=='funny':
    val.iloc[i,3]=1
  if val.iloc[i,3]=='not_funny':
    val.iloc[i,3]=0

  if val.iloc[i,4]=='very_twisted':
    val.iloc[i,4]=3
  if val.iloc[i,4]=='twisted_meaning':
    val.iloc[i,4]=2  
  if val.iloc[i,4]=='general':
    val.iloc[i,4]=1
  if val.iloc[i,4]=='not_sarcastic':
    val.iloc[i,4]=0    

  if val.iloc[i,5]=='hateful_offensive':
    val.iloc[i,5]=3
  if val.iloc[i,5]=='very_offensive':
    val.iloc[i,5]=2  
  if val.iloc[i,5]=='slight':
    val.iloc[i,5]=1
  if val.iloc[i,5]=='not_offensive':
    val.iloc[i,5]=0    

  if val.iloc[i,6]=='motivational':
    val.iloc[i,6]=1
  if val.iloc[i,6]=='not_motivational':
    val.iloc[i,6]=0    

Setting target values for the model depending on the task being solved out of A, B, C (https://competitions.codalab.org/competitions/20629)

In [ ]:
cls='offensive' #possible values : humour	sarcasm	offensive	motivational	overall_sentiment
data_train[1]=train[cls]
data_val[1]=val[cls]

Upsampling Data to handle class imbalance

In [ ]:
UpSample=True

if UpSample:
  from imblearn.under_sampling import RandomUnderSampler
  from imblearn.over_sampling import RandomOverSampler
  #=RandomUnderSampler(random_state=42,).fit_resample(X,Y)
  #X,Y=RandomUnderSampler(random_state=42).fit_resample(data_train.iloc[:,0][:,np.newaxis],data_train.iloc[:,1])
  X,Y=RandomOverSampler(random_state=42).fit_resample(data_train.iloc[:,[0,2]],data_train.iloc[:,1])
  data_train=pd.concat((pd.DataFrame(X),pd.DataFrame(Y)),axis=1)
  data_train.columns=['text','image','class']
  data_val.columns=['text','class','image']
  data_test.columns=['text','waste','image']
  data_train

else:
  data_train.columns=['text','class','image']
  data_val.columns=['text','class','image'] 
  data_test.columns=['text','waste','image']
  data_train  

Setting maximum sequence length for the inputs to RoBERTa

In [ ]:
max_seq_length=0
for i in tqdm(range(data_train.shape[0])):
    max_seq_length=max(len(data_train.iloc[i,0].split()),max_seq_length)
max_seq_length=max_seq_length+2     
print(max_seq_length)

# output : 
# 201

Textual data processing function to prepare meme overlay text for input to Roberta Model

In [ ]:
def convert_examples_to_features(sentences, label_list, max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    input_ids, input_masks, segment_ids, labels = [], [], [], []
    for index in tqdm_notebook(range(len(sentences)),desc="Converting examples to features"):
        sentence = sentences[index] #example.text_a.split(' ')
        if sentence=='':
          sentence=" "
        input_id = tokenizer.encode_plus(sentence,max_length=max_seq_length,pad_to_max_length=True)['input_ids']
        input_mask = tokenizer.encode_plus(sentence,max_length=max_seq_length,pad_to_max_length=True)['attention_mask']
        segment_id = tokenizer.encode_plus(sentence,max_length=max_seq_length,pad_to_max_length=True)['token_type_ids']
        label = label_list[index]
        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
        labels.append(label)
    return (
        np.array(input_ids),
        np.array(input_masks),
        np.array(segment_ids),
        np.array(labels)
    )


Processing textual data using above function

In [ ]:
from transformers import *
tokenizer=RobertaTokenizer.from_pretrained('roberta-base')
#tokenizer=AlbertTokenizer.from_pretrained("albert-base-v2")

data_train=data_train.reset_index(drop=True)
data_val=data_val.reset_index(drop=True)

(train_input_ids, train_input_masks, train_segment_ids, train_labels 
) = convert_examples_to_features(data_train['text'],data_train['class'],max_seq_length,tokenizer)

(val_input_ids, val_input_masks, val_segment_ids, val_labels 
) = convert_examples_to_features(data_val['text'],data_val['class'],max_seq_length,tokenizer)

(test_input_ids, test_input_masks, test_segment_ids, test_labels 
) = convert_examples_to_features(data_test['text'],np.ones(data_test.shape[0]),max_seq_length,tokenizer)


Defining F1 metric

In [ ]:
from tensorflow.keras import backend as K

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

Keras Callbacks to simplify model saving

In [ ]:
class Saver(tf.keras.callbacks.Callback):
  def on_train_begin(self,logs={}):
    self.score=0
  def on_epoch_end(self,logs={},*args):
    #self.model.save_weights('/media/data_dump/Pradyumna/empha/model-{}-{}-{}-{}-L.h5'.format(lr,epochs,dropout,layers))  
    predictions=self.model.predict([val_input_ids, val_input_masks, val_segment_ids])
    res=f1_score(val_labels,np.argmax(predictions,axis=1),average='macro')
    print("f1_score=",res)
    if res>self.score:
        self.score=res
        self.model.save_weights('/content/drive/My Drive/test-2.h5')      

Defining model architecture and training

In [ ]:
from tensorflow.keras.layers import Input,Dense,Bidirectional,LSTM


# Best hyperparameters
lr=1e-5
epochs=15
layer=0
dropout=0



token_inputs = tf.keras.layers.Input(shape=(None,), name='word_inputs', dtype=tf.int32)
mask_inputs = tf.keras.layers.Input(shape=(None,), name='mask_inputs', dtype=tf.int32)
seg_inputs = tf.keras.layers.Input(shape=(None,), name='seg_inputs', dtype=tf.int32)

inputs=[token_inputs,mask_inputs,seg_inputs]

transformer_outputs= TFRobertaModel.from_pretrained('roberta-base')(inputs)[0][:,0,:]

step=transformer_outputs


if layer>=3:
  step=tf.keras.layers.Dense(512,activation='relu')(step)
  if dropout!=0:
      step=tf.keras.layers.Dropout(rate=dropout)(step)
if layer>=2:
  step=tf.keras.layers.Dense(256,activation='relu')(step)
  if dropout!=0:
      step=tf.keras.layers.Dropout(rate=dropout)(step)
if layer>=1:    
  step=tf.keras.layers.Dense(64,activation='relu')(step)
  if dropout!=0:
      step=tf.keras.layers.Dropout(rate=dropout)(step)    


pred=tf.keras.layers.Dense(3,activation='softmax')(step)

model=tf.keras.Model(inputs=inputs,outputs=pred)


model.compile(loss='sparse_categorical_crossentropy',
      optimizer=tf.keras.optimizers.Adam(lr=lr),
      metrics=[f1,'accuracy'])
model.summary()

#from sklearn.utils.class_weight import compute_class_weight
#class_weights = compute_class_weight('balanced', np.unique(data_train.iloc[:,1]), data_train.iloc[:,1])

model.fit([train_input_ids,train_input_masks,train_segment_ids],
      train_labels,
      epochs=epochs,
      batch_size=32,
      validation_data=([val_input_ids,val_input_masks,val_segment_ids],val_labels),
      #class_weight=dict(enumerate(class_weights))  
      #callbacks=[
          #tf.keras.caltlbacks.LearningRateScheduler(scheduler,verbose=1)
          #TensorBoardColabCallback(tbc)
          #Saver()
      #  ] 
        )
predictions=model.predict([val_input_ids, val_input_masks, val_segment_ids])
print(f1_score(val_labels, 
            np.argmax(predictions,axis=1),average='macro'))
print(f1_score(val_labels, 
            np.argmax(predictions,axis=1),average='micro'))

print("Done!!")  

Final test predictions for submission

In [ ]:
predictions=model.predict([test_input_ids, test_input_masks, test_segment_ids])
prediction=np.argmax(predictions,axis=1)

Making submission file based on the task (A,B,C) being solved 

In [ ]:
with open('answer.txt',"w") as f:
  for i in range(len(prediction)):
    if prediction[i]==2:
      f.write(str(-1))
    else:
      f.write(str(prediction[i]))
    f.write("_9999_9999")
    f.write("\n")    